In [1]:
import training
import preprocessing as pre
import evaluation

SyntaxError: invalid syntax (training.py, line 14)

In [ ]:
# No notebook
data_path = '../data/new_logs_labels.csv'
X, y = pre.load_data(data_path)
y.head()

In [ ]:
X = X.fillna('missing')


In [ ]:
test_size = 0.2  # 80% for training, 20% for testing
X_train, X_test, y_train, y_test = pre.split_train_test_data(X, y, test_size, random_state=42)


In [ ]:
import pandas as pd

print("Nro de instancias de cada classe em y_train:\n")
print(pd.Series(y_train).value_counts())
print("\n\nNro de instancias de cada classe em y_test:\n")
print(pd.Series(y_test).value_counts())

In [ ]:
non_numeric_cols_train = X_train.select_dtypes(exclude=['float', 'int']).columns
non_numeric_cols_test = X_test.select_dtypes(exclude=['float', 'int']).columns

print("Non-numeric columns in X_train:")
print(non_numeric_cols_train)

print("\nNon-numeric columns in X_test:")
print(non_numeric_cols_test)

In [ ]:
X_train, label_encoders = pre.encode_categorical_columns(X_train)
X_test = pre.apply_encoders_to_test_data(X_test, label_encoders)

In [ ]:
import pandas as pd

print("Nro de instancias de cada classe em y_train:\n")
print(pd.Series(y_train).value_counts())
print("\n\nNro de instancias de cada classe em y_test:\n")
print(pd.Series(y_test).value_counts())

In [ ]:
y_train, label_encoder = pre.encode_single_column(y_train)
y_test = label_encoder.transform(y_test)

In [ ]:
X_train_over, y_train_over = pre.apply_smote(X_train, y_train)

In [ ]:
import pandas as pd

print("Nro de instancias de cada classe em y_train:\n")
print(pd.Series(y_train_over).value_counts())


In [ ]:
import pandas as pd

# Visualizar os tipos das colunas de X_train_over
print("Tipos das colunas de X_train_over:")
x_train_types = X_train_over.dtypes

# Visualizar os tipos das colunas de X_test
print("\nTipos das colunas de X_test:")
X_test_types = X_test.dtypes

In [ ]:
# Pré-processar os dados uma vez
preprocessor = pre.create_preprocessor(X_train_over)
X_train_preprocessed = preprocessor.fit_transform(X_train_over)
X_test_preprocessed = preprocessor.transform(X_test)

In [ ]:
print(X_train_preprocessed[:5])

In [ ]:
print(X_train_preprocessed.shape)
import pandas as pd

# Supondo que 'X_train_preprocessed' seja seu numpy.ndarray
df = pd.DataFrame(X_train_preprocessed)

# Agora você pode chamar .describe() no DataFrame
print(df.describe())

In [ ]:
# Geração dos relatórios
feature_names = X_train.columns  # Assumindo que os nomes das características são as colunas
print("feature_names: ", feature_names)

In [ ]:
# Verificação antes de chamar a função de treinamento
print(f"X_train_preprocessed shape: {X_train_preprocessed.shape}")
print(f"y_train shape: {y_train_over.shape}")

In [ ]:
cv = 10
n_iter = 100

# Escolher a métrica de avaliação
scoring_metric = 'roc_auc_ovr'  # Pode ser 'f1_macro', 'balanced_accuracy', 'roc_auc_ovr', etc.

# Chamar o treinamento com otimização bayesiana
trained_models = training.train_model(
    X_train_preprocessed, y_train_over, training.BAYESIAN_OPTIMIZATION, n_iter=n_iter, cv=cv, scoring=scoring_metric
)

In [ ]:
# Caminhos
dirpath = "../output/"
model_dir = "../models/"

# Geração dos relatórios
feature_names = X_train.columns  # Assumindo que os nomes das características são as colunas
reports = evaluation.generate_reports(trained_models, X_train_preprocessed, y_train_over, X_test_preprocessed, y_test, feature_names)

# Impressão dos relatórios
print(evaluation.print_reports(reports, dirpath))

evaluation.save_reports_to_csv(reports, dirpath)

# Salvar todos os modelos
saved_models = evaluation.dump_all_models(trained_models, model_dir)
print("Modelos salvos:", saved_models)